<a href="https://colab.research.google.com/github/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/blob/main/08_nlp_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/refs/heads/main/helper_functions/helper_functions.py

--2025-10-26 05:11:21--  https://raw.githubusercontent.com/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/refs/heads/main/helper_functions/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6733 (6.6K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   6.58K  --.-KB/s    in 0s      

2025-10-26 05:11:22 (88.1 MB/s) - ‘helper_functions.py’ saved [6733/6733]



In [2]:
from helper_functions import unzip_data,plot_loss_curves,compare_historys,create_tensorboard_callback

In [3]:
# Get the Dataset from kaggle NLP with Disaster Tweets
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2025-10-26 05:11:26--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.170.207, 74.125.200.207, 74.125.130.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.170.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   726KB/s    in 0.8s    

2025-10-26 05:11:27 (726 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
unzip_data("nlp_getting_started.zip")

In [5]:
# Become One with Dataset (Visualize)
import pandas as pd
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [8]:
# Shuffle Dataset
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
# What does Test Dataset Look like
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
# How many examples of each classes
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [11]:
# How many Total Samples
len(train_df),len(test_df)

(7613, 3263)

# **Visualize some random training examples**

In [12]:
import random
random_index=random.randint(0,len(train_df)-5)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
    _,text,target=row
    print(f'Target : {target} \n ',"(real disaster)" if target>0 else "not real disaster")
    print(f'Text : \n {text}')
    print('---\n')

Target : 1 
  (real disaster)
Text : 
 Disaster control teams are studying ways to evacuate the port area in response to tidal wave warnings.[900037]
---

Target : 1 
  (real disaster)
Text : 
 MH370: Reunion debris is from missing Malaysia flight http://t.co/6iMe8KJaCV
---

Target : 1 
  (real disaster)
Text : 
 when you don't know which way an ambulance is coming from &lt;&lt;
---

Target : 1 
  (real disaster)
Text : 
 #Japan marks 70th anniversary of #Hiroshima atomic bombing (from @AP) http://t.co/qREInWg0GS
---

Target : 0 
  not real disaster
Text : 
 I'm in bed eating a twister and drinking a cup of tea. I am not dunking the twister in the tea. That would be well weird.
---



In [13]:
#Splitting data into Train and Validation
from sklearn.model_selection import train_test_split

train_sentances,val_sentances,train_labels,val_labels=train_test_split(
                                                      train_df_shuffled['text'].to_numpy(),
                                                      train_df_shuffled['target'].to_numpy(),
                                                      test_size=0.1,
                                                      random_state=42)

In [14]:
# Check lengths
len(train_sentances),len(val_sentances),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

# **Convert Text into Numbers**

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
text_vectorizer=TextVectorization(
                                  max_tokens=10000, # Set max_tokens to a value
                                  standardize='lower_and_strip_punctuation',
                                  split='whitespace',
                                  ngrams=None,
                                  output_mode='int',
                                  output_sequence_length=None,
                                  pad_to_max_tokens=True,
                                   )

In [16]:
max_vocab_length=100   # Max number of words to have in vocabulary
max_length=15  # Max length our sequence will be
text_vectorizer=TextVectorization(max_tokens=max_vocab_length,
                                  standardize='lower_and_strip_punctuation',
                                  split='whitespace',
                                  ngrams=None,
                                  output_mode='int',
                                  output_sequence_length=max_vocab_length,
                                  pad_to_max_tokens=True)

In [17]:
# Fit the Text Vectorizer
text_vectorizer.adapt(train_sentances)

In [18]:
# Create a sample sentance and tokenize it
sample_sentance=random.choice(train_sentances)
print(f' The Original Sentance \n {sample_sentance} \n The Text Vectorized Version \n')
text_vectorizer([sample_sentance])

 The Original Sentance 
 Obama Declares Disaster for Typhoon-Devastated Saipan: Obama signs disaster declaration for Northern Marians a... http://t.co/XDt4VHFn7B 
 The Text Vectorized Version 



<tf.Tensor: shape=(1, 100), dtype=int64, numpy=
array([[ 1,  1, 75, 10,  1,  1,  1,  1, 75,  1, 10,  1,  1,  3,  1,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0]])>

In [19]:
# Get uniqe words in vocab
words_in_vocab=text_vectorizer.get_vocabulary()
top_5_words=words_in_vocab[:5]
bottom_5_words=words_in_vocab[-5:]
print(f'The Number of words in vocab are : {len(words_in_vocab)}')
print(f'The Five most common words in vocab are : {top_5_words}')
print(f'The Five least words in vocab are : {bottom_5_words}')

The Number of words in vocab are : 100
The Five most common words in vocab are : ['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
The Five least words in vocab are : [np.str_('buildings'), np.str_('rt'), np.str_('first'), np.str_('cant'), np.str_('see')]


In [20]:
# Creating a embedding layer that turns text vectorized into the embeding vectors
from tensorflow.keras.layers import Embedding

embedding=Embedding(input_dim=max_vocab_length,
                    output_dim=128,
                    )

In [21]:
# Get Random Sentance from trainning set
random_sentance=random.choice(train_sentances)
print(f'Original Text \n {random_sentance}\n')
sample_embedding=embedding(text_vectorizer([random_sentance]))
sample_embedding

Original Text 
 'Among other main factors behind pedestrian fatalities are people failing to yield to a car' http://t.co/dgUL7FfJt2



<tf.Tensor: shape=(1, 100, 128), dtype=float32, numpy=
array([[[-0.04694747,  0.02119812,  0.04551835, ...,  0.03751495,
          0.04793226, -0.03888023],
        [-0.04694747,  0.02119812,  0.04551835, ...,  0.03751495,
          0.04793226, -0.03888023],
        [-0.04694747,  0.02119812,  0.04551835, ...,  0.03751495,
          0.04793226, -0.03888023],
        ...,
        [ 0.04370045, -0.00997155, -0.00665887, ...,  0.04970368,
          0.03441938, -0.02893131],
        [ 0.04370045, -0.00997155, -0.00665887, ...,  0.04970368,
          0.03441938, -0.02893131],
        [ 0.04370045, -0.00997155, -0.00665887, ...,  0.04970368,
          0.03441938, -0.02893131]]], dtype=float32)>

# ***Lets Now Create our Baseline Model***
its common to use ML model first or non DL model and then switch to DL model

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0=Pipeline([
    ('tfid',TfidfVectorizer()),
    ('clf',MultinomialNB()),
])

model_0.fit(train_sentances,train_labels)
baseline_score=model_0.score(val_sentances,val_labels)

print(f'Our Baseline Model Acheives Accuracy of : {baseline_score*100:.2f}')

Our Baseline Model Acheives Accuracy of : 79.27


In [23]:
# Model Predictions
baseline_preds=model_0.predict(val_sentances)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

# **Creating an evaluation function**

In [24]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support


def calculate_results(y_true,y_pred):
  model_accuracy=accuracy_score(y_true=y_true,y_pred=y_pred)
  model_precission,model_recall,model_f1,_=precision_recall_fscore_support(y_true=y_true,y_pred=y_pred,average='weighted')

  model_results={
     "accuracy":model_accuracy,
     "precsion":model_precission,
     "recall":model_recall,
     'f1_score':model_f1
  }
  return model_results

In [25]:
# Get Baseline Results
baseline_results=calculate_results(y_true=val_labels,y_pred=baseline_preds)

baseline_results

{'accuracy': 0.7926509186351706,
 'precsion': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

# **Model1 : Feed Forward Neural Network ( Dense Model)**

In [26]:
from helper_functions import create_tensorboard_callback
from tensorflow.keras import layers

In [27]:
SAVE_DIR="ModelLogs"

inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x) # Add this layer to reduce dimensions
outputs=layers.Dense(1,activation='sigmoid')(x)
model_1=tf.keras.Model(inputs,outputs,name="model_1_Dense")
model_1.summary()

Model: "model_1_Dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,929 (50.50 KB)

 Trainable params: 12,929 (50.50 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_1_history=model_1.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels),callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name='model_1_dense')])

Saving TensorBoard log files to: ModelLogs/model_1_dense/20251026-051132
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5585 - loss: 0.6852 - val_accuracy: 0.5433 - val_loss: 0.6891
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5824 - loss: 0.6696 - val_accuracy: 0.5538 - val_loss: 0.6654
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6129 - loss: 0.6551 - val_accuracy: 0.5997 - val_loss: 0.6618
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6342 - loss: 0.6393 - val_accuracy: 0.6168 - val_loss: 0.6339
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6650 - loss: 0.6140 - val_accuracy: 0.5984 - val_loss: 0.6504


In [29]:
model_1.evaluate(val_sentances,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5845 - loss: 0.6762


[0.6504325866699219, 0.5984252095222473]

In [30]:
model_1_predict_probs=model_1.predict(val_sentances)
model_1_predict_probs

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[0.38164508],
       [0.50552285],
       [0.5709377 ],
       [0.2088908 ],
       [0.29879102],
       [0.47246432],
       [0.5520835 ],
       [0.49718314],
       [0.50798285],
       [0.17032097],
       [0.33674362],
       [0.37448415],
       [0.42232364],
       [0.31745845],
       [0.4612574 ],
       [0.33480594],
       [0.18072245],
       [0.2501111 ],
       [0.27232024],
       [0.35553023],
       [0.16210772],
       [0.08505306],
       [0.26442254],
       [0.3025895 ],
       [0.34582725],
       [0.45189524],
       [0.08239754],
       [0.37387925],
       [0.15248094],
       [0.18799847],
       [0.29824454],
       [0.3030188 ],
       [0.38563755],
       [0.31884646],
       [0.37533957],
       [0.09321768],
       [0.47603378],
       [0.16495675],
       [0.15591675],
       [0.52402693],
       [0.23220763],
       [0.4461039 ],
       [0.26792246],
       [0.35950264],
       [0.23571001],
       [0.4233403 ],
       [0.29642612],
       [0.597

In [31]:
model_1_predict_probs.shape

(762, 1)

In [32]:
model_1_predict_probs[0]

array([0.38164508], dtype=float32)

In [33]:
# Make the preictions comaparable with the actaul labels
model_1_predict_probs=tf.squeeze(tf.round(model_1_predict_probs))
model_1_predict_probs[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 0., 1., 0., 1., 0.], dtype=float32)>

In [34]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

# ***Calculate Model Results***

In [35]:
model_1_results=calculate_results(y_true=val_labels,y_pred=model_1_predict_probs)
model_1_results

{'accuracy': 0.5984251968503937,
 'precsion': 0.6658484491058051,
 'recall': 0.5984251968503937,
 'f1_score': 0.5205395273408879}

In [36]:
import numpy as np
np.array(list(model_1_results.values()))>np.array(list(baseline_results.values()))

array([False, False, False, False])

# ***Lets Visualize the embedding that Model learned using Tensorflow Projector***

In [37]:
words_in_vocab=text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(100,
 ['',
  '[UNK]',
  np.str_('the'),
  np.str_('a'),
  np.str_('in'),
  np.str_('to'),
  np.str_('of'),
  np.str_('and'),
  np.str_('i'),
  np.str_('is')])

In [38]:
model_1.summary()

Model: "model_1_Dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,789 (151.52 KB)

 Trainable params: 12,929 (50.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 25,860 (101.02 KB)

In [39]:
embed_weights=model_1.get_layer('embedding').get_weights()[0]
embed_weights,print(embed_weights.shape)

(100, 128)


(array([[ 0.01365811,  0.03091951,  0.02005928, ...,  0.08388501,
         -0.0280317 , -0.06804219],
        [ 0.01129229, -0.15195915, -0.12734437, ..., -0.14716743,
          0.22710441,  0.14358094],
        [ 0.1662095 ,  0.16329436,  0.16955568, ...,  0.1489195 ,
         -0.12611312, -0.16335775],
        ...,
        [-0.03640025, -0.05898622, -0.0791558 , ..., -0.0255525 ,
          0.04710792,  0.03099846],
        [ 0.25467154,  0.34615955,  0.4275126 , ...,  0.2903726 ,
         -0.4177854 , -0.29285625],
        [ 0.14889567,  0.3005157 ,  0.33518073, ...,  0.24990274,
         -0.23999597, -0.2619192 ]], dtype=float32),
 None)

Write the weights to disk. To use the Embedding Projector, you will upload two files in tab separated format: a file of vectors (containing the embedding), and a file of meta data (containing the words).  This code is from the tensorflow documentation Here[https://www.tensorflow.org/text/guide/word_embeddings]

In [40]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [41]:
# After this download step use projector tensorflow to visualzize it
"""
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass
"""


"\ntry:\n  from google.colab import files\n  files.download('vectors.tsv')\n  files.download('metadata.tsv')\nexcept Exception:\n  pass\n"

# ***MODEL2 =  LSTM [LONG SHORT TERM MEMORY]***

In [52]:
from tensorflow.keras import layers


In [53]:
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.LSTM(64,return_sequences=True)(x)
x=layers.LSTM(64)(x)
x=layers.Dense(64,activation='relu')(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_2=tf.keras.Model(inputs,outputs)

In [54]:
model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [55]:
model_2.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 100, 64)        │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 99,457 (388.50 KB)

 Trainable params: 99,457 (388.50 KB)

 Non-trainable params: 0 (0.00 B)

In [56]:
model_2_history=model_2.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_2_LSTM")])

Saving TensorBoard log files to: ModelLogs/model_2_LSTM/20251026-053945
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.5819 - loss: 0.6836 - val_accuracy: 0.5433 - val_loss: 0.6899
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5767 - loss: 0.6823 - val_accuracy: 0.5433 - val_loss: 0.6908
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.5636 - loss: 0.6854 - val_accuracy: 0.5433 - val_loss: 0.6915
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5707 - loss: 0.6836 - val_accuracy: 0.5433 - val_loss: 0.6906
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5645 - loss: 0.6854 - val_accuracy: 0.5433 - val_loss: 0.6930


In [57]:
model_2_pred_probs=model_2.predict(val_sentances)
model_2_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


array([[0.41464683],
       [0.41464683],
       [0.41464683],
       [0.41464683],
       [0.41464683],
       [0.41464683],
       [0.41464683],
       [0.41464683],
       [0.41464683],
       [0.41464683]], dtype=float32)

In [59]:
model_2_preds=tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>